In [1]:
import yfinance as yf
import pandas as pd
import mysql.connector

df = yf.download("ZC=F", start="2015-01-01", end="2024-12-31")
df.reset_index(inplace=True)

df = df[['Date', 'Open', 'High', 'Low', 'Close', 'Volume']]
df.columns = ['date', 'open', 'high', 'low', 'close', 'volume']

df['volume'] = df['volume'].fillna(0)
df[['open', 'high', 'low', 'close']] = df[['open', 'high', 'low', 'close']].fillna(0)
df = df.drop_duplicates(subset=['date'])

df['date'] = pd.to_datetime(df['date']).dt.date

df['volume'] = df['volume'].apply(lambda x: int(x))
df['open'] = df['open'].apply(lambda x: float(x))
df['high'] = df['high'].apply(lambda x: float(x))
df['low'] = df['low'].apply(lambda x: float(x))
df['close'] = df['close'].apply(lambda x: float(x))

conn = mysql.connector.connect(
    host='database-1.cba44miw26by.ap-northeast-2.rds.amazonaws.com',
    user='psy',
    password='0000',
    database='pig'
)
cursor = conn.cursor()

cursor.execute("DROP TABLE IF EXISTS corn_futures")
cursor.execute("""
    CREATE TABLE corn_futures (
        date DATE PRIMARY KEY,
        open FLOAT,
        high FLOAT,
        low FLOAT,
        close FLOAT,
        volume INT
    )
""")


data = [(
    row['date'],
    float(row['open']),
    float(row['high']),
    float(row['low']),
    float(row['close']),
    int(row['volume'])
) for _, row in df.iterrows()]


insert_sql = """
    INSERT INTO corn_futures (date, open, high, low, close, volume)
    VALUES (%s, %s, %s, %s, %s, %s)
"""
cursor.executemany(insert_sql, data)

conn.commit()
cursor.close()
conn.close()

print("옥수수 선물 데이터가 MySQL에 성공적으로 삽입되었습니다!")


YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


옥수수 선물 데이터가 MySQL에 성공적으로 삽입되었습니다!


In [2]:
import yfinance as yf
import pandas as pd
import mysql.connector


df = yf.download("USDKRW=X", start="2015-01-01", end="2024-12-31")
df.reset_index(inplace=True)

df = df[['Date', 'Open', 'High', 'Low', 'Close']]
df.columns = ['date', 'open', 'high', 'low', 'close']

df[['open', 'high', 'low', 'close']] = df[['open', 'high', 'low', 'close']].fillna(0)

df = df.drop_duplicates(subset=['date'])

df['date'] = pd.to_datetime(df['date']).dt.date


df['open'] = df['open'].apply(lambda x: float(x))
df['high'] = df['high'].apply(lambda x: float(x))
df['low'] = df['low'].apply(lambda x: float(x))
df['close'] = df['close'].apply(lambda x: float(x))


conn = mysql.connector.connect(
    host='database-1.cba44miw26by.ap-northeast-2.rds.amazonaws.com',
    user='psy',
    password='0000',
    database='pig'
)
cursor = conn.cursor()


cursor.execute("DROP TABLE IF EXISTS exchange_rate")
cursor.execute("""
    CREATE TABLE exchange_rate(
        date DATE PRIMARY KEY,
        open FLOAT,
        high FLOAT,
        low FLOAT,
        close FLOAT
    )
""")


data = [(
    row['date'],
    float(row['open']),
    float(row['high']),
    float(row['low']),
    float(row['close'])
) for _, row in df.iterrows()]

insert_sql = """
    INSERT INTO exchange_rate (date, open, high, low, close)
    VALUES (%s, %s, %s, %s, %s)
"""
cursor.executemany(insert_sql, data)


conn.commit()
cursor.close()
conn.close()

print("USD/KRW 환율 데이터가 MySQL에 성공적으로 삽입되었습니다")

[*********************100%***********************]  1 of 1 completed


USD/KRW 환율 데이터가 MySQL에 성공적으로 삽입되었습니다


In [3]:
import mysql.connector
import pandas as pd

# MySQL 연결
conn = mysql.connector.connect(
    host='database-1.cba44miw26by.ap-northeast-2.rds.amazonaws.com',
    user='psy',
    password='0000',
    database='pig'
)
cursor = conn.cursor()

# 연간 평균 데이터를 가져오는 쿼리
query = """
    SELECT cf.year, cf.corn_close AS corn_price, er.usdkrw_close AS exchange_rate, 
           cf.corn_close * er.usdkrw_close AS krw_based_price
    FROM (
        SELECT DATE_FORMAT(date, '%Y') AS year, AVG(close/100) AS corn_close 
        FROM corn_futures 
        GROUP BY DATE_FORMAT(date, '%Y')
    ) cf
    JOIN (
        SELECT DATE_FORMAT(date, '%Y') AS year, AVG(close) AS usdkrw_close
        FROM exchange_rate
        GROUP BY DATE_FORMAT(date, '%Y')
    ) er
    ON cf.year = er.year;
"""

# 쿼리 실행
cursor.execute(query)

# 결과를 데이터프레임으로 변환
rows = cursor.fetchall()
columns = ['year', 'corn_price', 'exchange_rate', 'krw_based_price']
corn_df = pd.DataFrame(rows, columns=columns)

# MySQL 연결 종료
cursor.close()
conn.close()

# 데이터 출력
print(corn_df)

   year  corn_price  exchange_rate  krw_based_price
0  2015    3.766726    1129.992490      4256.372308
1  2016    3.584540    1158.687008      4153.359926
2  2017    3.592241    1129.928255      4058.974645
3  2018    3.680663    1099.446478      4046.691590
4  2019    3.833879    1164.024425      4462.728761
5  2020    3.633300    1178.353741      4281.313112
6  2021    5.819851    1143.850536      6657.039906
7  2022    6.938855    1291.037695      8958.322827
8  2023    5.637070    1305.279728      7357.953196
9  2024    4.236863    1361.472228      5768.370695


In [4]:
import pandas as pd

# 엑셀 파일 경로
file_path = '/home/sungyoon/dev_ws/eda/data/월별 년도별 배합사료가격(00년_25년) ~2월 (1).xls'

# 모든 시트를 딕셔너리 형태로 읽어오기
sheets = pd.read_excel(file_path, sheet_name=None)

# 각 시트를 데이터프레임으로 접근
for sheet_name, df in sheets.items():
    print(f"Sheet name: {sheet_name}")
    print(df)

Sheet name: 년도별가격동향
    ○ 월별 배합사료(공장도) 가격 동향  Unnamed: 1  Unnamed: 2 Unnamed: 3 Unnamed: 4  \
0                    NaN         NaN         NaN        NaN        NaN   
1               1. 2000년         NaN         NaN        NaN        NaN   
2                    NaN         NaN         NaN        NaN        NaN   
3                    축종별          1월          2월         3월         4월   
4                    양계용      262.31      260.51     263.24     263.87   
..                   ...         ...         ...        ...        ...   
281                  젖소용  620.023714  619.311934        NaN        NaN   
282                  고기소  518.059303  518.428237        NaN        NaN   
283                 기  타  628.171495   636.34927        NaN        NaN   
284                 오  리  596.223963  604.819945        NaN        NaN   
285            양축용\n가중평균  610.056803  611.851742        NaN        NaN   

    Unnamed: 5 Unnamed: 6 Unnamed: 7 Unnamed: 8 Unnamed: 9 Unnamed: 10  \
0          NaN   

In [5]:
import pandas as pd

file_path = "/home/sungyoon/dev_ws/eda/data/월별 년도별 배합사료가격(00년_25년) ~2월.xlsx"


target_sheets = [
    '15년 월별가격 ',
    '16년 월별가격)',
    '17년 월별가격',
    '18년 월별가격 ',
    '19년 월별가격 ',
    '20년 월별가격',
    '21년 월별가격',
    '22년 월별가격',
    '23년 월별가격',
    '24년 월별가격',
]


yangdon_rows = {}

for sheet in target_sheets:
    df = pd.read_excel(file_path, sheet_name=sheet)
   
    mask = df.astype(str).apply(lambda row: row.str.contains('양돈', na=False).any(), axis=1)
    filtered = df[mask]
    if not filtered.empty:
        yangdon_rows[sheet] = filtered


result_df = pd.concat(yangdon_rows.values(), keys=yangdon_rows.keys())

print(result_df)


            Unnamed: 0 Unnamed: 1  Unnamed: 2  Unnamed: 3  Unnamed: 4  \
15년 월별가격  9         양돈        NaN      606.27      610.81      610.39   
16년 월별가격) 9         양돈        NaN       565.4       560.4       556.9   
17년 월별가격  9         양돈        NaN       550.7       548.1       549.4   
18년 월별가격  9         양돈        NaN      537.04      541.16      540.25   
19년 월별가격  9         양돈        NaN      559.04      553.95      562.54   
20년 월별가격  9         양돈        NaN      567.94      568.85      563.79   
21년 월별가격  9         양돈        NaN  565.587014  568.653918   580.63616   
22년 월별가격  7         양돈        NaN      670.68       673.5  695.971403   
23년 월별가격  6        NaN        NaN  790.221949  787.458778  784.467472   
24년 월별가격  6        NaN        NaN  733.529865  727.154539  732.708137   

             Unnamed: 5  Unnamed: 6  Unnamed: 7  Unnamed: 8  Unnamed: 9  \
15년 월별가격  9      597.93      596.44      596.61      600.38      574.08   
16년 월별가격) 9       558.7       563.1       557.

In [6]:
import re

long_format_rows = []

for sheet in target_sheets:
    df = pd.read_excel(file_path, sheet_name=sheet)


    mask = df.astype(str).apply(lambda row: row.str.contains('양돈', na=False).any(), axis=1)
    filtered = df[mask]
    if filtered.empty:
        continue


    match = re.search(r"(\d+)", sheet)
    if not match:
        continue
    year_suffix = match.group(1)
    year = int("20" + year_suffix) if int(year_suffix) < 30 else int("19" + year_suffix)

    
    month_cols = filtered.columns[3:15] 
    prices = filtered.iloc[0, 2:14].values 

    for month_index, price in enumerate(prices, start=1):
        month_str = f"{year}-{month_index:02d}"
        long_format_rows.append({'month': month_str, 'feed_price': price})


feed_df = pd.DataFrame(long_format_rows)


print(feed_df.head(14))

      month  feed_price
0   2015-01      606.27
1   2015-02      610.81
2   2015-03      610.39
3   2015-04      597.93
4   2015-05      596.44
5   2015-06      596.61
6   2015-07      600.38
7   2015-08      574.08
8   2015-09      579.71
9   2015-10      570.51
10  2015-11      560.04
11  2015-12      559.73
12  2016-01      565.40
13  2016-02      560.40


In [7]:
corn_df['month'] = corn_df['year'] + '-01'  # Assuming you want to use January for each year
df = pd.merge(corn_df, feed_df, on='month')

print(df)

   year  corn_price  exchange_rate  krw_based_price    month  feed_price
0  2015    3.766726    1129.992490      4256.372308  2015-01  606.270000
1  2016    3.584540    1158.687008      4153.359926  2016-01  565.400000
2  2017    3.592241    1129.928255      4058.974645  2017-01  550.700000
3  2018    3.680663    1099.446478      4046.691590  2018-01  537.040000
4  2019    3.833879    1164.024425      4462.728761  2019-01  559.040000
5  2020    3.633300    1178.353741      4281.313112  2020-01  567.940000
6  2021    5.819851    1143.850536      6657.039906  2021-01  565.587014
7  2022    6.938855    1291.037695      8958.322827  2022-01  670.680000
8  2023    5.637070    1305.279728      7357.953196  2023-01  790.221949
9  2024    4.236863    1361.472228      5768.370695  2024-01  733.529865


In [8]:
df = pd.merge(corn_df, feed_df, on = 'month')

print(df)

   year  corn_price  exchange_rate  krw_based_price    month  feed_price
0  2015    3.766726    1129.992490      4256.372308  2015-01  606.270000
1  2016    3.584540    1158.687008      4153.359926  2016-01  565.400000
2  2017    3.592241    1129.928255      4058.974645  2017-01  550.700000
3  2018    3.680663    1099.446478      4046.691590  2018-01  537.040000
4  2019    3.833879    1164.024425      4462.728761  2019-01  559.040000
5  2020    3.633300    1178.353741      4281.313112  2020-01  567.940000
6  2021    5.819851    1143.850536      6657.039906  2021-01  565.587014
7  2022    6.938855    1291.037695      8958.322827  2022-01  670.680000
8  2023    5.637070    1305.279728      7357.953196  2023-01  790.221949
9  2024    4.236863    1361.472228      5768.370695  2024-01  733.529865


In [9]:
import mysql.connector
import pandas as pd

# MySQL 연결 설정
conn = mysql.connector.connect(
    host='database-1.cba44miw26by.ap-northeast-2.rds.amazonaws.com',
    user='psy',
    password='0000',
    database='pig'
)
cursor = conn.cursor()

# 기존 테이블 삭제 및 새 테이블 생성
cursor.execute("DROP TABLE IF EXISTS feed_price")
cursor.execute("""
    CREATE TABLE feed_price(
        year DATE PRIMARY KEY,
        corn_price FLOAT,
        exchange_rate FLOAT,
        krw_based_price FLOAT,
        feed_price FLOAT
    )
""")

# 데이터 정리
df = df.dropna(subset=['year', 'corn_price', 'exchange_rate', 'krw_based_price', 'feed_price'])
df = df.drop_duplicates(subset=['year'])
df['year'] = pd.to_datetime(df['year'], errors='coerce')
df = df.dropna(subset=['year'])
df.rename(columns={'KRW-based price': 'krw_based_price'}, inplace=True)
df['year'] = df['year'].dt.to_period('M').dt.to_timestamp()

# 데이터 준비
data = [(
    row['year'].strftime('%Y-%m-%d'),  # '%Y-%m' -> '%Y-%m-%d'
    float(row['corn_price']),
    float(row['exchange_rate']),
    float(row['krw_based_price']),
    float(row['feed_price'])
) for _, row in df.iterrows()]

# 데이터 삽입
insert_sql = """
    INSERT INTO feed_price (year, corn_price, exchange_rate, krw_based_price, feed_price)
    VALUES (%s, %s, %s, %s, %s)
"""
cursor.executemany(insert_sql, data)

# 커밋 및 연결 종료
conn.commit()
cursor.close()
conn.close()

print("사료 가격 데이터가 MySQL에 성공적으로 삽입되었습니다.")

사료 가격 데이터가 MySQL에 성공적으로 삽입되었습니다.


In [ ]:
import yfinance as yf
from datetime import datetime, timedelta

def get_previous_trading_day(ticker_symbol, max_lookback=7):
    today = datetime.today()
    for i in range(1, max_lookback + 1):
        day = today - timedelta(days=i)
        data = yf.Ticker(ticker_symbol).history(start=day.strftime('%Y-%m-%d'), end=(day + timedelta(days=1)).strftime('%Y-%m-%d'))
        if not data.empty:
            return day.strftime('%Y-%m-%d'), data['Close'].iloc[0]
    return None, None

symbol = "ZC=F"
date_str, close_price = get_previous_trading_day(symbol)


print(f"{date_str} 기준 옥수수 선물 종가: {close_price:.2f} USD")


2025-03-24 기준 옥수수 선물 종가: 464.50 USD
